## [Interactive Analysis with the Spark Shell](https://spark.apache.org/docs/latest/quick-start.html#interactive-analysis-with-the-spark-shell)

### [Basic](https://spark.apache.org/docs/latest/quick-start.html#basics)

ENG: Spark’s primary abstraction is a distributed collection of items called a Dataset. Datasets can be created from Hadoop InputFormats (such as HDFS files) or by transforming other Datasets.

TR: Spark' ın `birincil soyutlaması(primary abstraction)` `Dataset`lerdir. `Dataset` dediği şey ise okuduğu dosyadaki `item` olarak kabul ettiği şeylerin kümesidir. `Dataset` HDSF dosyalarından ya da farklı kaynaklardan da oluşturulabilir. `distributed collection of items` bu ifade de ki `distributed` ifadesi ne olabilir bilmiyorum.

In [2]:
val textFile = spark.read.textFile("../README.md") //Scala dizininde bulunan README.md dosyasını okuyor.
// Text dosyasından yeni bir Dataset oluşturuyor.

// Bir Dataset' ten value alma işlemine action deniyor.
// Bir Dataset' ten yeni bir Dataset oluşturmaya da transform deniyor.

textFile: org.apache.spark.sql.Dataset[String] = [value: string]


In [7]:
// Aşağıdaki bir action işlemidir.
textFile.count() // Number of items in this Dataset.
// README.md satırları bu örnekte item' dır

res4: Long = 21


In [9]:
// Action
textFile.first() // First item in this Dataset

res6: String = ## Açıklama


In [10]:
// Transform: Bir Dataset' ten yeni bir Dataset oluşturma işlemine denir.
// Elimizdeki Dataset'ten yeni bir Dataset oluşturmak için filter isimli bir fonksiyon çağıracağız.
// filter fonksiyonu ile elimizdeki Dataset' in item' larının alt kümesinden yeni bir Dataset oluşturacağız.
val lineWithSpark = textFile.filter(line => line.contains("Spark")) 
// line => line.contains("Spark") -> Bu kısım Python' daki lambda fonksiyonu ile aynıdır.
// line isimli parametre alan bir fonksiyon var ve bu fonksiyon içinde "Spark" geçen item' lardan yeni bir Dataset
// oluşturuyor.

lineWithSpark: org.apache.spark.sql.Dataset[String] = [value: string]


In [15]:
// Action
println("Number of lines contains Spark: " + lineWithSpark.count())

// Transform ve action işlemlerini bu şekilde birleştirebiliyoruz.
textFile.filter(line => line.contains("Spark")).count()

// Jupyter Notebook' ta hücrenin en sonundaki satır out kısmında yazılır yani print edilmesine gerek yoktur.
// Üst satırlardaki işlemler ise print edilmedilir. Bu sebeple ilk satırda print varken ikincisinde yoktur.

Number of lines contains Spark: 3


res11: Long = 3


### [More on Dataset Operations](https://spark.apache.org/docs/latest/quick-start.html#more-on-dataset-operations)

Action ve Transform işlemlerini kullanarak daha karmaşık hesaplamalar yapılabilir. Mesela elimizdeki Dataset' teki en çok kelime barındıran satırın kaç kelime barındırdığını bulmak istersek:

In [3]:
textFile.map(line => line.split(" ").size).reduce((a, b) => if (a > b) a else b)
// Bu işlemi teker teker açıklamak istiyorum

res1: Int = 12


In [6]:
// map(line => line.split(" ").size)

/*
    map: Kendisine parametre olarak verilen fonksiyonu Dataset' teki bütün item' lara uygular
    ve Dataset' ten yeni bir Dataset oluşturur. Yeni oluşturduğu Dataset' in item değerleri
    olarak kendisine parametre olarak verilen fonksiyonun çıktısını yazar.

    Map fonksiyonuna parametre olarak verilen fonksiyon sayıları 2 ile çarpsın. `1, 2, 3` değerlerini
    item olarak tutan Dataset' in map fonksiyonu çıktısı: `2, 4, 6` olur (1 * 2, 2 * 2, 3 * 2).
*/

/*
    Anonymous Functions: Python' daki lambda function' un Scala karşılığıdır. Bilmeyenler için
    kısa ve isimsiz bir şekilde fonksiyon tanımlama yoludur.

    val sum = (num1: Int, num2: Int) => num1 + num2

    Okun sol tarafı fonksiyonun parametreleridir. Sağ tarafı ise return değeridir.

    sum(5, 6) => çıktı 11' dir.
*/

/*
    line => line.split(" ").size

    Yukarıdaki Anonymous Function kendisine parametre olarak verilen String ifadeyi split fonksiyonu ile
    parçalayıp split fonksiyonunun döndürdüğü Array' in size' ını yani boyutunu return eder.

*/

val textFileLineWordCount = textFile.map(line => line.split(" ").size)

println(textFileLineWordCount) // textFileLineWordCount Dataset' indeki item' ların type' ları int' tir. Out kısmında bakın.

textFileLineWordCount.show(numRows=5)
// textFile Dataset' inde item değerleri String' tir(textFile' ı okuduğumuz cell' e bakın).
// Bu String tipi Spark' a özel bir String tipi değildir.
// Scala' nın String type' ıdır.

[value: int]
+-----+
|value|
+-----+
|    2|
|    1|
|    6|
|    1|
|    5|
+-----+
only showing top 5 rows



textFileLineWordCount: org.apache.spark.sql.Dataset[Int] = [value: int]


In [4]:
// reduce((a, b) => if (a > b) a else b)

/*
    reduce: map gibi parametre olarak fonksiyon alır. Parametre olarak aldığı fonksiyonun iki parametresi ve tek return' ü
    olmalıdır.

    reduce kelime anlamı olarak azaltmak demek. Dataset' teki item' ları ikişer olarak ele alıp belirli işlemleri
    uygulamaya yarar. Max, Min vb. işlemlerdir. reduce işlemini tam olarak nasıl yapar bilmiyorum.

*/

/*
    (a, b) => if (a > b) a else b

    Yukarıdaki ifade bir Anonymous Function'dır ve kendisine parametre olarak verilen iki değerden en büyüğünü
    return eder.
*/

// reduce((a, b) => if (a > b) a else b)
// Yukarıdaki işlem item' ları ikişer ikişer karşılaştırarak en büyük değer olanı elde etmeye yarar.

textFile.map(line => line.split(" ").size).reduce((a, b) => if (a > b) a else b)

res2: Int = 12


In [18]:
import java.lang.Math

textFile.map(line => line.split(" ").size).reduce((a,b) => Math.max(a, b))
// Okunurluğu ya da verimliliği arttırmak için Scala dilinin destekleklediği her şey kullanılabilir.
// Math.max

import java.lang.Math
res13: Int = 22


In [7]:
// One common data flow pattern is MapReduce, as popularized by Hadoop. Spark can implement MapReduce flows easily
val wordCounts = textFile.flatMap(line => line.split(" ")).groupByKey(identity).count()

// Yukarıdaki işlemi teker teker anlatmak istiyorum.

wordCounts: org.apache.spark.sql.Dataset[(String, Long)] = [key: string, count(1): bigint]


In [12]:
// flatMap(line => line.split(" "))

/*
    flatMap: map fonksiyonu ile farkı şu: Dataset' in kaç item' ı varsa map fonksiyonunun çıktısı olan Dataset' de
    o kadar item' a sahiptir. flatMap' te ise ya aynı sayıda item' a ya da daha fazla sayıda item' a sahip olur.

    Eğer flatMap' e parametre olarak verilen fonksiyonun çıktısı tek bir değer dönüyorsa map gibi çalışır fakat
    birden fazla değer dönüyorsa dönülen her değer yeni Dataset' te item olur.
    
*/


textFile.map(line => line.split(" ")).show(numRows=5) // map
val flatTextFile = textFile.flatMap(line => line.split(" ")) //flatMap
flatTextFile.show(numRows=5)

+--------------------+
|               value|
+--------------------+
|      [##, Açıklama]|
|                  []|
|[ScalaSpark, ile,...|
|                  []|
|[###, Klasör, isi...|
+--------------------+
only showing top 5 rows

+----------+
|     value|
+----------+
|        ##|
|  Açıklama|
|          |
|ScalaSpark|
|       ile|
+----------+
only showing top 5 rows



flatTextFile: org.apache.spark.sql.Dataset[String] = [value: string]


In [12]:
// groupByKey(identity)

/*
    identity: identity bir Scala fonksiyonudur. Kendine parametre olarak verilen değeri return eder.
    
    identity(5) => 5 döner.
*/

// groupByKey ifadesini örnek ile anlatacağım.

In [11]:
// İlk önce bir Dataset oluşturalım. Aşağıdaki gibi bir Dataset oluşturduğumuzda item' ın tipi
// Dataset[Row] olur. Yukarıdaki örnekte ise Dataset[String]' ti. Bu bilgi önemli.

// Data oluşturma
val columns = Seq("class" ,"number")
val data = Seq(
    ("A", -5), ("A", -5), ("A", -5), ("B", -4), ("C", -3), ("C", -3), ("D", -2), ("F", -1), 
    ("H", 0), ("H", 0), ("H", 0), ("H", 0), ("H", 0), ("K", 1), ("K", 1), ("K", 1), ("K", 1), 
    ("L", 2), ("L", 2), ("M", 3), ("S", 4), ("T", 5)
)

val df = spark.sparkContext.parallelize(data).toDF(columns:_*) // _* ne demek bilmiyorum.
df.show(numRows=5)
// Cell' in Out kısmına bakarsanız df' in DataFrame olduğunu görürsünüz. DataFrame demek Dataset[Row] demektir.

val groupedDf = df.groupByKey(row => (row.getInt(1)).abs + row.getString(0)).count()
/*
    row => (row.getInt(1)).abs + row.getString(0)

    ("A", -5) item' ımız olsun. Bu işlemde yapılan her şey Dataset' teki bütün item' lara uygulanır.

    row.getInt(1).abs => {
        row.getInt(1) bu değer -5 değer
        abs ile -5 değerinin mutlak değeri alınır yani 5 olur
    }
    NOT: getInt olmasının sebebi Dataset' teki item' lar Row nesnesi cinsinden tutulduğu içindir. 1 değerinin
    anlamı ise Int değerinin Row' un `1.` kolonunda olmasından.

    row.getString(0) => {
        A değeri çekilmiş olur.
    }

    Sonuç olarak 5A değeri elde edilir ve bu değer yeni Dataset' in item değeri olarak yazılır. Kolon adı ise
    key olur. Cell' deki ikinci çıktıya bakınız.

    Yukarıdaki işlem bütün item' lara uygulanır ve elde edilen Dataset' üzerinde key kolonu için groupBy işlemi uygulanır.
*/

/*
    count: count bir aggregation işlemidir ve gruplanan ifadelerin kaç tane olduğunu bize verir.
*/

groupedDf.show(numRows=5) // Görmek için ikinci çıktıya bakınız.

+-----+------+
|class|number|
+-----+------+
|    A|    -5|
|    A|    -5|
|    A|    -5|
|    B|    -4|
|    C|    -3|
+-----+------+
only showing top 5 rows

+---+--------+
|key|count(1)|
+---+--------+
| 5A|       3|
| 4B|       1|
| 3C|       2|
| 2D|       1|
| 1F|       1|
+---+--------+
only showing top 5 rows



columns: Seq[String] = List(class, number)
data: Seq[(String, Int)] = List((A,-5), (A,-5), (A,-5), (B,-4), (C,-3), (C,-3), (D,-2), (F,-1), (H,0), (H,0), (H,0), (H,0), (H,0), (K,1), (K,1), (K,1), (K,1), (L,2), (L,2), (M,3), (S,4), (T,5))
df: org.apache.spark.sql.DataFrame = [class: string, number: int]
groupedDf: org.apache.spark.sql.Dataset[(String, Long)] = [key: string, count(1): bigint]


In [14]:
val wordCounts = textFile.flatMap(line => line.split(" ")).groupByKey(identity).count()
// Yukarıdaki işlem groupBy ile de yapabilir lakin böyle bir şey tercih edilmiş.

/*
Yukarıdaki işlemde ise flatMap fonksiyonun kendisi tek kolonlu bir Dataset döndürür: Dataset[String]. Bu Dataset' e DatasetA diyeceğim.

Yukarıdaki işlem => {
    DatasetA' daki bütün item' lara teker teker identity fonksiyonu uygulanır ve yeni bir Dataset elde edilir: DatasetB
    DatasetB' de key kolonuna göre gruplanıp count alınır.
}
*/


wordCounts: org.apache.spark.sql.Dataset[(String, Long)] = [key: string, count(1): bigint]


In [15]:
wordCounts.collect() // Dataset' i bir Array' a dönüştüren fonksiyon.

res7: Array[(String, Long)] = Array((Okuduğum,1), ([Jupyter,1), ([Read,1), (dökümanlar,1), ([install](scala/install),1), (code,1), (işlemler:,1), (*,7), (Interactive,1), (--------,1), (with,1), (barındıran,1), (RDD,1), (Analysis,1), (Kurulumlar,1), (Programming,1), (yazma:,1), (açıklama,1), (klasör.,1), (notebook',1), (okuma,,1), (doc,1), (Shell](scala/doc/scala_doc_01.ipynb),1), (dosyaları,1), (|,5), ([SparkDoc:,2), (------,1), (Linkler,1), (Başka,1), (iligli,1), (the,1), (ScalaSpark,1), (ve,1), (Belirli,1), (kaynak](https://medium.com/@bogdan.cojocar/how-to-run-scala-and-spark-in-the-jupyter-notebook-328a80090b3b),1), ([code](scala/code),1), (yazma,1), (and,1), (Scala,1), (barındırdıkları,1), (JSON,1), ([doc](scala/doc),1), (ile,1), (###,1), (install,1), (##,2), (klasör,1), (ta,1), (S...


### [Caching](https://spark.apache.org/docs/latest/quick-start.html#caching)

Bu kısmı tam olarak anlamadım. Araştırıp detaylı olarak yazmayı düşünüyorum

In [ ]:
// Self-Contained Applications diye bir kısım vardı. Ona bakmadım.